In [1]:
import numpy as np
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
%matplotlib inline

Reading the datasets that we have. We will need movies and the ratings files. 

In [2]:
movie = pd.read_csv('Netflix_Dataset_Movie.csv')
rating= pd.read_csv('Netflix_Dataset_Rating(CUT).csv')

Quick view of the two dataframes, the rows and the columns

In [3]:
movie.head()

,Movie_ID,Year,Name
0,1,2003,Dinosaur Planet
1,2,2004,Isle of Man TT 2004 Review
2,3,1997,Character
3,4,1994,Paula Abdul's Get Up & Dance
4,5,2004,The Rise and Fall of ECW


In [4]:
rating.head()

,User_ID,Rating,Movie_ID
0,712664,5,3
1,1331154,4,3
2,2632461,3,3
3,44937,5,3
4,656399,4,3


Using pandas' replace function, we remove the year from the title in the movie dataframe and add a separate year column.
This is done using regular expressions. Find a year stored between parentheses. We specify the parantheses so we don't conflict with movies that have years in their titles.

First we create the new Year column in the movie dataframe.

Now removing the year from the title in the Title column. 

Collaborative filtering doesn't recommend based on the features of the movie. The recommendation is based on the likes and dislikes or ratings of the neighbours or other users. 
So we will drop the genre column, since there is no use of it. 

In [5]:
movie.drop(columns=['Year'], inplace=True)

In [6]:
movie.head()

,Movie_ID,Name
0,1,Dinosaur Planet
1,2,Isle of Man TT 2004 Review
2,3,Character
3,4,Paula Abdul's Get Up & Dance
4,5,The Rise and Fall of ECW


Now, coming to the ratings dataframe, we have the movieId column that is common with the movie dataframe. Each user has given multiple ratings for different movies. The column Timestamp is not needed for the recommendation system. So we can drop it. 

The technique is called Collaborative Filtering, which is also known as User-User Filtering. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the Pearson Correlation Function.

The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

In [7]:
user = [
            {'Name':'Dinosaur Planet', 'rating':3},
            {'Name':'The Rise and Fall of ECW', 'rating':3},
            {'Name':'Screamers', 'rating':3},
            {'Name':'Immortal Beloved', 'rating':2.5},
            {'Name':'Strange Relations', 'rating':5}
         ] 
inputMovie = pd.DataFrame(user)
inputMovie

,Name,rating
0,Dinosaur Planet,3.0
1,The Rise and Fall of ECW,3.0
2,Screamers,3.0
3,Immortal Beloved,2.5
4,Strange Relations,5.0


We need to now add the movieId column from the movie dataframe into the inputMovie Dataframe. 
First filter out the rows that contain the input movies' title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [8]:
#Filtering out the movies by Name
Id = movie[movie['Name'].isin(inputMovie['Name'].tolist())]
print(Id)
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovie = pd.merge(Id, inputMovie)
inputMovie

    Movie_ID                      Name
0          1           Dinosaur Planet
4          5  The Rise and Fall of ECW
15        16                 Screamers
17        18          Immortal Beloved
20        21         Strange Relations


,Movie_ID,Name,rating
0,1,Dinosaur Planet,3.0
1,5,The Rise and Fall of ECW,3.0
2,16,Screamers,3.0
3,18,Immortal Beloved,2.5
4,21,Strange Relations,5.0


**Finding the users who have seen the same movies from the rating dataframe**
With the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [9]:
#Filtering out users that have watched movies that the input has watched and storing it
users = rating[rating['Movie_ID'].isin(inputMovie['Movie_ID'].tolist())]
users.head()

,User_ID,Rating,Movie_ID
10903,554875,2,16
10904,2480084,3,16
10905,1794216,3,16
10906,1276913,5,16
10907,400164,4,16


**Finding the users who have seen the same movies from the rating dataframe**
With the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [10]:
users.shape

(11936, 3)

In [11]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
user_SubsetGroup = users.groupby(['User_ID'])

In [12]:
#showing one such group example by getting all the users of a particular userId
user_SubsetGroup.get_group

<bound method BaseGroupBy.get_group of <pandas.core.groupby.generic.DataFrameGroupBy object at 0x00000249AAAD5280>>

In [13]:
#Sorting it so users with movie most in common with the input will have priority
user_SubsetGroup = sorted(user_SubsetGroup,  key=lambda x: len(x[1]), reverse=True)

In [14]:
user_SubsetGroup

[(10374,
         User_ID  Rating  Movie_ID
  12197    10374       4        16
  22425    10374       4        18),
 (15402,
         User_ID  Rating  Movie_ID
  11948    15402       3        16
  21535    15402       2        18),
 (16272,
         User_ID  Rating  Movie_ID
  12423    16272       3        16
  23218    16272       4        18),
 (17890,
         User_ID  Rating  Movie_ID
  13228    17890       2        16
  26241    17890       4        18),
 (21705,
         User_ID  Rating  Movie_ID
  11229    21705       3        16
  18810    21705       3        18),
 (21823,
         User_ID  Rating  Movie_ID
  13356    21823       3        16
  26755    21823       3        18),
 (35996,
         User_ID  Rating  Movie_ID
  12583    35996       2        16
  23781    35996       4        18),
 (39242,
         User_ID  Rating  Movie_ID
  11616    39242       3        16
  20282    39242       3        18),
 (39297,
         User_ID  Rating  Movie_ID
  11906    39297       3    

**Similarity of users to input user**
Next, we are going to compare all users  to our specified user and find the one that is most similar.
we're going to find out how similar each user is to the input through the Pearson Correlation Coefficient. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation") 

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

In [15]:
user_SubsetGroup = user_SubsetGroup[0:10]
user_SubsetGroup


[(10374,
         User_ID  Rating  Movie_ID
  12197    10374       4        16
  22425    10374       4        18),
 (15402,
         User_ID  Rating  Movie_ID
  11948    15402       3        16
  21535    15402       2        18),
 (16272,
         User_ID  Rating  Movie_ID
  12423    16272       3        16
  23218    16272       4        18),
 (17890,
         User_ID  Rating  Movie_ID
  13228    17890       2        16
  26241    17890       4        18),
 (21705,
         User_ID  Rating  Movie_ID
  11229    21705       3        16
  18810    21705       3        18),
 (21823,
         User_ID  Rating  Movie_ID
  13356    21823       3        16
  26755    21823       3        18),
 (35996,
         User_ID  Rating  Movie_ID
  12583    35996       2        16
  23781    35996       4        18),
 (39242,
         User_ID  Rating  Movie_ID
  11616    39242       3        16
  20282    39242       3        18),
 (39297,
         User_ID  Rating  Movie_ID
  11906    39297       3    

In [16]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorDict = {}

#For every user group in our subset
for name, group in user_SubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='Movie_ID')
    inputMovie = inputMovie.sort_values(by='Movie_ID')
    #Get the N for the formula
    n = len(group)
    #Get the review scores for the movies that they both have in common
    temp = inputMovie[inputMovie['Movie_ID'].isin(group['Movie_ID'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp['rating'].tolist()
    #put the current user group reviews in a list format
    tempGroupList = group['Rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(n)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(n)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(n)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorDict[name] = 0


In [17]:
pearsonCorDict.items()

dict_items([(10374, 0), (15402, 1.0), (16272, -1.0), (17890, -1.0), (21705, 0), (21823, 0), (35996, -1.0), (39242, 0), (39297, 0), (44807, -1.0)])

In [18]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorDict, orient='index')
print(pearsonDF)
pearsonDF.columns = ['similarityIndex']
pearsonDF['User_ID'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

         0
10374  0.0
15402  1.0
16272 -1.0
17890 -1.0
21705  0.0
21823  0.0
35996 -1.0
39242  0.0
39297  0.0
44807 -1.0


,similarityIndex,User_ID
0,0.0,10374
1,1.0,15402
2,-1.0,16272
3,-1.0,17890
4,0.0,21705


In [19]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head(20)

,similarityIndex,User_ID
1,1.0,15402
0,0.0,10374
4,0.0,21705
5,0.0,21823
7,0.0,39242
8,0.0,39297
2,-1.0,16272
3,-1.0,17890
6,-1.0,35996
9,-1.0,44807


**Rating of selected users to all movies**
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our pearsonDF from the ratings dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [20]:
topUsersRating=topUsers.merge(rating, left_on='User_ID', right_on='User_ID', how='inner')
topUsersRating.head(15)

,similarityIndex,User_ID,Rating,Movie_ID
0,1.0,15402,3,16
1,1.0,15402,3,17
2,1.0,15402,2,18
3,1.0,15402,3,28
4,1.0,15402,3,44
5,1.0,15402,4,58
6,1.0,15402,3,77
7,1.0,15402,3,108
8,1.0,15402,2,122
9,1.0,15402,5,143


multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

In [21]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['Rating']
topUsersRating.head()

,similarityIndex,User_ID,Rating,Movie_ID,weightedRating
0,1.0,15402,3,16,3.0
1,1.0,15402,3,17,3.0
2,1.0,15402,2,18,2.0
3,1.0,15402,3,28,3.0
4,1.0,15402,3,44,3.0


In [22]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('Movie_ID').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
Movie_ID,,
3,-1.0,-4.0
16,-3.0,-5.0
17,0.0,0.0
18,-3.0,-15.0
26,0.0,0.0


In [23]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['Movie_ID'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,Movie_ID
Movie_ID,,
3,4.000000,3
16,1.666667,16
17,NaN,17
18,5.000000,18
26,NaN,26


In [24]:
recommendation_df.values

array([[4.00000000e+00, 3.00000000e+00],
       [1.66666667e+00, 1.60000000e+01],
       [           nan, 1.70000000e+01],
       ...,
       [           nan, 4.49200000e+03],
       [           nan, 4.49300000e+03],
       [3.00000000e+00, 4.49600000e+03]])

In [25]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,Movie_ID
Movie_ID,,
2595,inf,2595
2712,inf,2712
3624,inf,3624
353,inf,353
2874,inf,2874
3713,inf,3713
2866,inf,2866
3224,inf,3224
305,inf,305


In [26]:
movie.loc[movie['Movie_ID'].isin(recommendation_df.head(10)['Movie_ID'].tolist())]

,Movie_ID,Name
304,305,Jack
352,353,Life or Something Like It
1843,1844,Casualties of War
2594,2595,The Vanishing
2711,2712,The Hitcher
2865,2866,Message in a Bottle
2873,2874,Elf
3223,3224,All Quiet on the Western Front
3623,3624,The Last Samurai
3712,3713,Saw
